In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('https://shelterdata.s3.amazonaws.com/shelter1000.csv')

In [3]:
data['name'] = data['Name'].str.replace("\*","")

# separate the "sex upon outcome" column into property of an animal (male or female)
#and property of an outcome (was the animal spayed/neutered at the shelter or not)
data['sex'] = data['Sex upon Outcome'].replace({"Neutered Male":"M",
                                                "Intact Male":"M",
                                                "Intact Female":"F",
                                                "Spayed Female":"F",
                                                "Unknown":np.nan})

data['is_fixed'] = data['Sex upon Outcome'].replace({"Neutered Male":True,
                                                     "Intact Male":False,
                                                     "Intact Female":False,
                                                     "Spayed Female":True,
                                                     "Unknown":np.nan})

# prepare the data table for introducing the date dimension
# we'll use condensed date as the key, e.g. '20231021'
data['ts'] = pd.to_datetime(data.DateTime)
data['date_id'] = data.ts.dt.strftime('%Y%m%d')
data['time'] = data.ts.dt.time



# prepare th data table for introducing the outcome type dimension:
# introduce keys for the outcomes
outcomes_map = {'Rto-Adopt':1,
                'Adoption':2,
                'Euthanasia':3,
                'Transfer':4,
                'Return to Owner':5,
                'Died':6,
                'Disposal':7}
data['outcome_type_id'] = data['Outcome Type'].replace(outcomes_map)

data


<ipython-input-3-b8484960be48>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['name'] = data['Name'].str.replace("\*","")


,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,name,sex,is_fixed,ts,date_id,time,outcome_type_id
0,A794011,Chunk,05/08/2019 6:20:00 PM,May 2019,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,Chunk,M,True,2019-05-08 18:20:00,20190508,18:20:00,1
1,A776359,Gizmo,07/18/2018 4:02:00 PM,Jul 2018,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown,Gizmo,M,True,2018-07-18 16:02:00,20180718,16:02:00,2
2,A821648,NaN,08/16/2020 11:38:00 AM,Aug 2020,08/16/2019,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray,NaN,NaN,NaN,2020-08-16 11:38:00,20200816,11:38:00,3
3,A720371,Moose,02/13/2016 5:59:00 PM,Feb 2016,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,Moose,M,True,2016-02-13 17:59:00,20160213,17:59:00,2
4,A674754,NaN,03/18/2014 11:47:00 AM,Mar 2014,03/12/2014,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby,NaN,M,False,2014-03-18 11:47:00,20140318,11:47:00,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,A691208,NaN,11/11/2014 6:13:00 PM,Nov 2014,12/31/2013,Adoption,NaN,Cat,Spayed Female,10 months,Domestic Shorthair Mix,Black/White,NaN,F,True,2014-11-11 18:13:00,20141111,18:13:00,2
996,A718620,*Toby Sue,03/01/2016 1:53:00 PM,Mar 2016,07/23/2015,Adoption,NaN,Cat,Spayed Female,7 months,Domestic Shorthair Mix,Black/White,Toby Sue,F,True,2016-03-01 13:53:00,20160301,13:53:00,2
997,A705114,NaN,06/13/2015 3:47:00 PM,Jun 2015,06/11/2015,Transfer,Partner,Dog,Intact Male,2 days,German Shepherd Mix,Brown,NaN,M,False,2015-06-13 15:47:00,20150613,15:47:00,4
998,A800888,NaN,08/01/2019 3:27:00 PM,Aug 2019,07/03/2019,Transfer,Partner,Dog,Intact Female,4 weeks,Cavalier Span,Black/White,NaN,F,False,2019-08-01 15:27:00,20190801,15:27:00,4


In [4]:
animal_dim = data[['Animal ID','name','Date of Birth', 'sex', 'Animal Type', 'Breed', 'Color']]
animal_dim.columns = ['animal_id', 'name', 'dob', 'sex', 'animal_type', 'breed', 'color']
animal_dim.drop_duplicates()

,animal_id,name,dob,sex,animal_type,breed,color
0,A794011,Chunk,05/02/2017,M,Cat,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,07/12/2017,M,Dog,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,08/16/2019,NaN,Other,Raccoon,Gray
3,A720371,Moose,10/08/2015,M,Dog,Anatol Shepherd/Labrador Retriever,Buff
4,A674754,NaN,03/12/2014,M,Cat,Domestic Shorthair Mix,Orange Tabby
...,...,...,...,...,...,...,...
995,A691208,NaN,12/31/2013,F,Cat,Domestic Shorthair Mix,Black/White
996,A718620,Toby Sue,07/23/2015,F,Cat,Domestic Shorthair Mix,Black/White
997,A705114,NaN,06/11/2015,M,Dog,German Shepherd Mix,Brown
998,A800888,NaN,07/03/2019,F,Dog,Cavalier Span,Black/White


In [5]:
dates_dim = pd.DataFrame({
    'date_id':data.ts.dt.strftime('%Y%m%d'),
    'date':data.ts.dt.date,
    'year':data.ts.dt.year,
    'month':data.ts.dt.month,
    'day':data.ts.dt.day,
    })
dates_dim.drop_duplicates()

,date_id,date,year,month,day
0,20190508,2019-05-08,2019,5,8
1,20180718,2018-07-18,2018,7,18
2,20200816,2020-08-16,2020,8,16
3,20160213,2016-02-13,2016,2,13
4,20140318,2014-03-18,2014,3,18
...,...,...,...,...,...
994,20200117,2020-01-17,2020,1,17
995,20141111,2014-11-11,2014,11,11
996,20160301,2016-03-01,2016,3,1
997,20150613,2015-06-13,2015,6,13


In [6]:
outcome_types_dim = pd.DataFrame.from_dict(outcomes_map, orient='index').reset_index()
outcome_types_dim.columns=['outcome_type', 'outcome_type_id']
outcome_types_dim


,outcome_type,outcome_type_id
0,Rto-Adopt,1
1,Adoption,2
2,Euthanasia,3
3,Transfer,4
4,Return to Owner,5
5,Died,6
6,Disposal,7


In [7]:
outcomes_fct = data[["Animal ID", 'date_id','time','outcome_type_id','Outcome Subtype', 'is_fixed']]
outcomes_fct.rename(columns={"Animal ID":"animal_id", "Outcome Subtype":"outcome_subtype"})

,animal_id,date_id,time,outcome_type_id,outcome_subtype,is_fixed
0,A794011,20190508,18:20:00,1,NaN,True
1,A776359,20180718,16:02:00,2,NaN,True
2,A821648,20200816,11:38:00,3,NaN,NaN
3,A720371,20160213,17:59:00,2,NaN,True
4,A674754,20140318,11:47:00,4,Partner,False
...,...,...,...,...,...,...
995,A691208,20141111,18:13:00,2,NaN,True
996,A718620,20160301,13:53:00,2,NaN,True
997,A705114,20150613,15:47:00,4,Partner,False
998,A800888,20190801,15:27:00,4,Partner,False
